# SafeNSound: Distress Signal Detection ML Development

This notebook contains the machine learning development for the SafeNSound IoT-based distress signal detection system. The goal is to develop models that can accurately detect distress signals from audio data captured by sound sensors in semi-private rooms.

## Project Overview
- **Objective**: Detect distress signals (screams, calls for help, etc.) from ambient audio
- **Data**: Audio recordings from IoT sound sensors
- **Models**: Audio classification using deep learning and traditional ML approaches
- **Deployment**: Real-time inference on IoT devices

## 1. Import Essential Libraries

Import necessary libraries for audio processing, machine learning, and data visualization.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# Audio processing libraries
import librosa
import librosa.display
import soundfile as sf
from scipy import signal
from scipy.io import wavfile

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# Deep learning libraries (uncomment when needed)
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, LSTM, Dropout

print("Libraries imported successfully!")

## 2. Load and Explore Audio Dataset

Load audio files and create a dataset with labels (distress vs normal sounds). Explore the characteristics of the audio data.

In [ ]:
# Define data paths (update these paths to your actual data location)
DATA_PATH = "../data/"  # Update this path
DISTRESS_PATH = os.path.join(DATA_PATH, "distress/")
NORMAL_PATH = os.path.join(DATA_PATH, "normal/")

def load_audio_files(directory, label, max_files=None):
    """Load audio files from directory and return file paths with labels"""
    audio_data = []
    
    if not os.path.exists(directory):
        print(f"Warning: Directory {directory} does not exist. Creating sample data structure.")
        return []
    
    for i, filename in enumerate(os.listdir(directory)):
        if filename.endswith(('.wav', '.mp3', '.flac')):
            if max_files and i >= max_files:
                break
            file_path = os.path.join(directory, filename)
            audio_data.append({
                'file_path': file_path,
                'filename': filename,
                'label': label
            })
    
    return audio_data

# Load dataset (update paths as needed)
print("Loading audio dataset...")
distress_files = load_audio_files(DISTRESS_PATH, 'distress')
normal_files = load_audio_files(NORMAL_PATH, 'normal')

# Combine all files
all_files = distress_files + normal_files
dataset_df = pd.DataFrame(all_files)

print(f"Dataset loaded:")
print(f"- Total files: {len(all_files)}")
print(f"- Distress samples: {len(distress_files)}")
print(f"- Normal samples: {len(normal_files)}")

if len(all_files) > 0:
    print("\nDataset preview:")
    print(dataset_df.head())
    print(f"\nLabel distribution:")
    print(dataset_df['label'].value_counts())
else:
    print("\nNote: No audio files found. Please add your audio data to the appropriate directories.")
    print("Expected structure:")
    print("- data/distress/ (for distress signal audio files)")
    print("- data/normal/ (for normal ambient sound files)")

## 3. Audio Data Preprocessing and Feature Extraction

Extract meaningful features from audio files including MFCC, spectral features, and other audio characteristics for machine learning.

In [ ]:
def extract_audio_features(file_path, sr=22050, n_mfcc=13):
    """
    Extract comprehensive audio features from an audio file
    """
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=sr)
        
        # Basic audio properties
        duration = len(y) / sr
        
        # MFCC features (most important for speech/audio classification)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_std = np.std(mfcc, axis=1)
        
        # Spectral features
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0]
        
        # Energy and tempo features
        rms_energy = librosa.feature.rms(y=y)[0]
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        
        # Aggregate features
        features = {
            'duration': duration,
            'tempo': tempo,
            'spectral_centroid_mean': np.mean(spectral_centroids),
            'spectral_centroid_std': np.std(spectral_centroids),
            'spectral_rolloff_mean': np.mean(spectral_rolloff),
            'spectral_rolloff_std': np.std(spectral_rolloff),
            'spectral_bandwidth_mean': np.mean(spectral_bandwidth),
            'spectral_bandwidth_std': np.std(spectral_bandwidth),
            'zero_crossing_rate_mean': np.mean(zero_crossing_rate),
            'zero_crossing_rate_std': np.std(zero_crossing_rate),
            'rms_energy_mean': np.mean(rms_energy),
            'rms_energy_std': np.std(rms_energy),
        }
        
        # Add MFCC features
        for i, (mean_val, std_val) in enumerate(zip(mfcc_mean, mfcc_std)):
            features[f'mfcc_{i+1}_mean'] = mean_val
            features[f'mfcc_{i+1}_std'] = std_val
            
        return features
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def visualize_audio_sample(file_path, title="Audio Sample"):
    """Visualize waveform and spectrogram of an audio file"""
    try:
        y, sr = librosa.load(file_path, sr=22050)
        
        plt.figure(figsize=(15, 8))
        
        # Waveform
        plt.subplot(2, 2, 1)
        plt.title(f'{title} - Waveform')
        librosa.display.waveshow(y, sr=sr)
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        
        # Spectrogram
        plt.subplot(2, 2, 2)
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz')
        plt.title(f'{title} - Spectrogram')
        plt.colorbar(format='%+2.0f dB')
        
        # MFCC
        plt.subplot(2, 2, 3)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        librosa.display.specshow(mfcc, sr=sr, x_axis='time')
        plt.title(f'{title} - MFCC')
        plt.colorbar()
        
        # Spectral features over time
        plt.subplot(2, 2, 4)
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        frames = range(len(spectral_centroids))
        t = librosa.frames_to_time(frames)
        plt.plot(t, spectral_centroids, label='Spectral Centroid')
        plt.title(f'{title} - Spectral Centroid')
        plt.xlabel('Time (s)')
        plt.ylabel('Hz')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error visualizing {file_path}: {e}")

print("Audio processing functions defined successfully!")

## 4. Split Data into Training and Testing Sets

Extract features from all audio files and prepare the dataset for machine learning training.

In [ ]:
# Extract features from all audio files
if len(all_files) > 0:
    print("Extracting features from audio files...")
    
    features_list = []
    labels_list = []
    
    for i, file_info in enumerate(all_files):
        print(f"Processing file {i+1}/{len(all_files)}: {file_info['filename']}")
        
        features = extract_audio_features(file_info['file_path'])
        if features is not None:
            features_list.append(features)
            labels_list.append(file_info['label'])
    
    # Convert to DataFrame
    features_df = pd.DataFrame(features_list)
    features_df['label'] = labels_list
    
    print(f"\nFeature extraction completed!")
    print(f"Features shape: {features_df.shape}")
    print(f"Features extracted: {list(features_df.columns[:-1])}")
    
    # Display feature statistics
    print("\nFeature dataset preview:")
    print(features_df.head())
    
    # Prepare data for machine learning
    X = features_df.drop('label', axis=1)
    y = features_df['label']
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"\nData split completed:")
    print(f"Training set: {X_train_scaled.shape}")
    print(f"Test set: {X_test_scaled.shape}")
    print(f"Label classes: {label_encoder.classes_}")
    
else:
    print("No audio files found. Please add audio data to continue with model training.")
    # Create dummy data for demonstration
    print("Creating dummy dataset for demonstration...")
    np.random.seed(42)
    X_train_scaled = np.random.random((100, 38))  # 38 features typically extracted
    X_test_scaled = np.random.random((25, 38))
    y_train = np.random.choice([0, 1], 100)
    y_test = np.random.choice([0, 1], 25)
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.array(['distress', 'normal'])
    print("Dummy dataset created for demonstration purposes.")

## 5. Choose and Train Models

Train multiple machine learning models to find the best approach for distress signal detection.

In [ ]:
# Define and train multiple models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', random_state=42),
    'SVM Linear': SVC(kernel='linear', random_state=42)
}

trained_models = {}
model_scores = {}

print("Training models...")
for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train the model
    model.fit(X_train_scaled, y_train)
    trained_models[name] = model
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    model_scores[name] = {
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_scores': cv_scores
    }
    
    print(f"{name} - CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Display model comparison
print("\nModel Comparison (Cross-Validation Scores):")
print("-" * 50)
for name, scores in model_scores.items():
    print(f"{name:15s}: {scores['cv_mean']:.4f} (+/- {scores['cv_std'] * 2:.4f})")

# Select best model based on CV score
best_model_name = max(model_scores.keys(), key=lambda x: model_scores[x]['cv_mean'])
best_model = trained_models[best_model_name]

print(f"\nBest model: {best_model_name}")
print(f"Best CV score: {model_scores[best_model_name]['cv_mean']:.4f}")

## 6. Evaluate Model Performance

Assess model performance using various metrics and visualizations to understand how well the model detects distress signals.

In [ ]:
# Evaluate all models on test set
print("Evaluating models on test set...")
print("=" * 80)

for name, model in trained_models.items():
    print(f"\n{name} Results:")
    print("-" * 40)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    
    # Classification report
    print(f"\nDetailed Classification Report:")
    target_names = [f"Class {i} ({label})" for i, label in enumerate(label_encoder.classes_)]
    print(classification_report(y_test, y_pred, target_names=target_names))

# Confusion Matrix for best model
print(f"\nConfusion Matrix for {best_model_name}:")
y_pred_best = best_model.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(10, 6))

# Plot confusion matrix
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_, 
            yticklabels=label_encoder.classes_)
plt.title(f'Confusion Matrix - {best_model_name}')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Plot model comparison
plt.subplot(1, 2, 2)
model_names = list(model_scores.keys())
cv_means = [model_scores[name]['cv_mean'] for name in model_names]
cv_stds = [model_scores[name]['cv_std'] for name in model_names]

plt.bar(model_names, cv_means, yerr=cv_stds, capsize=5, alpha=0.7)
plt.title('Model Comparison (Cross-Validation)')
plt.ylabel('CV Score')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Feature importance (for Random Forest)
if 'Random Forest' in trained_models:
    rf_model = trained_models['Random Forest']
    feature_importance = rf_model.feature_importances_
    feature_names = X.columns
    
    # Sort features by importance
    indices = np.argsort(feature_importance)[::-1]
    
    plt.figure(figsize=(12, 8))
    plt.title('Feature Importance - Random Forest')
    plt.bar(range(min(20, len(feature_importance))), 
             feature_importance[indices[:20]], 
             align='center')
    plt.xticks(range(min(20, len(feature_importance))), 
               [feature_names[i] for i in indices[:20]], 
               rotation=45, ha='right')
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print(f"\nTop 10 Most Important Features:")
    for i in range(min(10, len(feature_importance))):
        print(f"{i+1:2d}. {feature_names[indices[i]]:25s}: {feature_importance[indices[i]]:.4f}")

## 7. Make Predictions and Model Deployment

Demonstrate how to use the trained model for real-time distress signal detection and prepare for IoT deployment.